In [15]:
import json
import os
from openai import OpenAI
from openai.types.chat import ChatCompletionMessage, ChatCompletionMessageToolCall
from typing import List, Dict, Any
from pprint import pprint

# Step 1: Set up the OpenAI client for Ollama
client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama's endpoint
    api_key="qwen3:8b",  # Dummy key for Ollama
)

# Step 2: Define a sample tool schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature to use",
                        "default": "fahrenheit",
                    },
                },
                "required": ["location"],
            },
        },
    },

{
    "type": "function",
    "function": {
        "name": "get_current_stock_prices",
        "description": "Get the current stock price for a given company ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., AAPL for Apple or TSLA for Tesla"
                },
                "exchange": {
                    "type": "string",
                    "description": "The stock exchange where the symbol is listed, e.g., NASDAQ or NYSE",
                    "default": "NASDAQ"
                },
                "currency": {
                    "type": "string",
                    "enum": ["USD", "EUR", "JPY"],
                    "description": "The currency to return the price in",
                    "default": "USD"
                }
            },
            "required": ["symbol"]
        }
    }
}

]

# Step 3: Define the tool execution function
def get_current_stock_prices(symbol: str, exchange: str = "NASDAQ", currency: str = "USD") -> str:
    """Sample tool function - returns hardcoded stock data as a string."""
    # In a real app, this would call a stock market API like Alpha Vantage, Yahoo Finance, etc.
    symbol = symbol.upper()
    
    dummy_prices = {
        "AAPL": 175.32,
        "TSLA": 265.78,
        "GOOGL": 135.20
    }
    
    if symbol in dummy_prices:
        price = dummy_prices[symbol]
        return f"Current price of {symbol} on {exchange}: {price} {currency}"
    else:
        return f"Stock data not available for {symbol} on {exchange} (currency: {currency})"

def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """Sample tool function - returns hardcoded weather data as a string."""
    # In a real app, this would call a weather API
    if "san francisco" in location.lower():
        return f"Weather in {location}: 72°F, sunny (unit: {unit})"
    else:
        return f"Weather data not available for {location} (unit: {unit})"


# Step 4: Initialize conversation history
messages: List[Dict[str, Any]] = [
    {"role": "user", "content": "Can you tell me the current weather in San Francisco? Also, how is Apple stock performing today? Do you think there's any correlation between the weather in San Francisco and Apple's stock price?"}
]

# Step 5: First API call - Send user message and tools to the model
try:
    response = client.chat.completions.create(
        model="qwen3:8b",  # Replace with your Ollama model (e.g., 'qwen3:8b')
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0,
    )
except Exception as e:
    print(f"Error in first API call: {e}")
    exit(1)

print("First Response")
print("=="*20)
pprint(response.model_dump())
# Step 6: Process the response
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print("Tool call detected:")
    # Add assistant's message (with tool calls) to history
    messages.append(
        {
            "role": response_message.role,
            "content": response_message.content,
            "tool_calls": [
                {
                    "id": tc.id,
                    "type": tc.type,
                    "function": {
                        "name": tc.function.name,
                        "arguments": tc.function.arguments,
                    },
                }
                for tc in tool_calls
            ],
        }
    )

    # Step 7: Execute tool calls
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        try:
            function_args = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as e:
            print(f"Error parsing tool call arguments: {e}")
            continue

        print(f"Executing tool: {function_name} with args: {function_args}")

        # Execute the tool
        if function_name == "get_current_weather":
            function_response = get_current_weather(
                location=function_args.get("location", ""),
                unit=function_args.get("unit", "fahrenheit"),
            )
        elif function_name == "get_current_stock_prices":
            function_response = get_current_stock_prices(
               **function_args
            )
        else:
            function_response = f"Error: Unknown function {function_name}"

        # Append tool response to messages
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": function_response,  # Ensure content is a string
            }
        )
    print("--"*20)
    pprint(messages)
    print("--"*20)
    # Step 8: Second API call - Send tool results back to the model
    try:
        second_response = client.chat.completions.create(
            model="qwen3:14b", 
            messages=messages,
        )
        final_message = second_response.choices[0].message.content
        print("\nFinal Response from Model:")
        print("=="*20)
        pprint(second_response.model_dump())
        print("=="*20)
        from rich.console import Console
        Console().print(f"[red]{final_message}[red]")
    except Exception as e:
        print(f"Error in second API call: {e}")
else:
    print("No tool was called.")
    print("Direct Response:", response_message.content)

First Response
{'choices': [{'finish_reason': 'tool_calls',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': None,
                          'audio': None,
                          'content': '<think>\n'
                                     "Okay, let's break down the user's query. "
                                     'They asked three things: the current '
                                     "weather in San Francisco, Apple's stock "
                                     "performance, and if there's a "
                                     'correlation between the weather and '
                                     "Apple's stock.\n"
                                     '\n'
                                     'First, I need to call the '
                                     'get_current_weather function for San '
                                     'Francisco. The parameters required are '
                                     "loc

<think>
Okay, so the user asked about the current weather in San Francisco, Apple's stock performance, and if there's a 
correlation between the two. Let me start by summarizing the information I received from the tool responses.

First, the weather in San Francisco is 72°F, which is equivalent to 22.2°C, and it's sunny. The stock price for 
Apple (AAPL) on the NASDAQ is $175.32 USD. Now, the user is asking if there's a correlation between the weather and
Apple's stock price.

Hmm, I know that stock prices are typically influenced by a variety of factors such as company performance, market 
trends, economic indicators, and investor sentiment. Weather conditions, while they can affect consumer behavior 
and business operations, are not usually a direct factor in stock performance, especially for a large company like 
Apple. 

Wait, but maybe there could be some indirect effects. For example, if there's a major weather event that impacts 
Apple's operations, like a severe storm affecting their retail stores or supply chain, that could have a temporary 
impact. However, San Francisco's weather being sunny and mild doesn't seem to have any immediate effect on Apple's 
stock. 

I should also consider that the user might be thinking about the morale of employees or consumer activity. A nice 
day might lead to more people shopping in Apple stores, but the stock market reacts to broader trends rather than 
daily weather. Plus, Apple is a global company, so the weather in one city is unlikely to have a significant impact
on their overall stock performance.

Another angle: sometimes, investors might react to news or events that are influenced by weather, but in this case,
the weather data provided doesn't indicate any such events. So, it's safe to say that there's no direct correlation
between the current weather in San Francisco and Apple's stock price. 

I should make sure to explain that while the tools can provide the current weather and stock price, the correlation
isn't something that can be determined through these tools alone. It would require a more in-depth analysis of 
historical data and market factors, which isn't covered by the functions available.
</think>

The current weather in San Francisco is **22.2°C (72°F)** with sunny conditions, and Apple's stock price is 
**$175.32 USD** on the NASDAQ. 

### Correlation Between Weather and Apple's Stock Price:
While the weather in San Francisco is pleasant today, **there is no direct or measurable correlation** between 
short-term weather conditions and Apple's stock price. Stock prices are influenced by factors such as:
- **Company performance** (e.g., quarterly earnings, product launches).
- **Market trends** (e.g., investor sentiment, sector performance).
- **Macroeconomic factors** (e.g., interest rates, inflation).
- **Global events** (e.g., geopolitical tensions, supply chain disruptions).

Weather in a single city is unlikely to impact a global company like Apple unless it directly affects operations 
(e.g., extreme weather disrupting retail stores or manufacturing). For now, the stock price appears unaffected by 
today's conditions.